In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

import torch
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
# from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_recall_curve, auc, accuracy_score, confusion_matrix
# from sklearn.naive_bayes import MultinomialNB

import matplotlib.pyplot as plt
import seaborn as sns # plotting problem

!pip install contractions
import contractions # expanding contractions in text can't -> cannot

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 9.9 MB/s eta 0:00:00
/kaggle/input/amazon-product-review-spam-and-non-spam/Home_and_Kitchen/Home_and_Kitchen.json
/kaggle/input/amazon-product-review-spam-and-non-spam/part.json/part.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Electronics/Electronics.json
/kaggle/input/amazon-product-review-spam-and-non-spam/separate.json/separate.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Clothing_Shoes_and_Jewelry/Clothing_Shoes_and_Jewelry.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Sports_and_Outdoors/Sports_and_Outdoors.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Cell_Phones_and_Accessories/Cell_Phones_and_Accessories.json
/kaggle/input/amazon-product-review-spam-and-non-spam/Toys_and_Games/Toys_and_Games.json


In [2]:
torch.backends.cudnn.deterministic = True
random.seed(21)
np.random.seed(21)
torch.manual_seed(21)
torch.cuda.manual_seed_all(21)

In [3]:
df = pd.read_json('/kaggle/input/amazon-product-review-spam-and-non-spam/Toys_and_Games/Toys_and_Games.json', lines=True).head(100000)
df = df.loc[:, ['reviewText', 'class']]
df

,reviewText,class
0,I love these felt nursery rhyme characters and...,1
1,I see no directions for its use. Therefore I h...,0
2,This is a great tool for any teacher using the...,1
3,"Great product, thank you! Our son loved the pu...",1
4,Although not as streamlined as the Algebra I m...,1
...,...,...
99995,Received this product in a timely fashion. I m...,0
99996,McFarlane Sports Series are fantastic and life...,1
99997,Fortune is a good figure. She has a very attra...,1
99998,I just thought that I'd jot a few words to let...,0


In [4]:
df['reviewText'] = df['reviewText'].apply(lambda x: contractions.fix(x))
df.loc[:, 'reviewText'] = df['reviewText'].str.lower()

# \W represents Special characters like "$" and "!!!"
df.loc[:, 'reviewText'] = df['reviewText'].str.replace('\W', ' ')

# \d represents Numeric digits like "19.99"
df.loc[:, 'reviewText'] = df['reviewText'].str.replace('\d', ' ')

df

,reviewText,class
0,i love these felt nursery rhyme characters and...,1
1,i see no directions for its use. therefore i h...,0
2,this is a great tool for any teacher using the...,1
3,"great product, thank you! our son loved the pu...",1
4,although not as streamlined as the algebra i m...,1
...,...,...
99995,received this product in a timely fashion. i m...,0
99996,mcfarlane sports series are fantastic and life...,1
99997,fortune is a good figure. she has a very attra...,1
99998,i just thought that i would jot a few words to...,0


In [5]:
y = df['class']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], y, test_size=0.33, random_state=21)

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve, auc


# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize input texts
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

# Convert labels to tensors
train_labels = torch.tensor(list(y_train))
test_labels = torch.tensor(list(y_test))

# Create datasets
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              train_labels)
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']),
                             test_labels)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 3)

# Training loop
model = torch.nn.DataParallel(model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

for epoch in range(3):  # Adjust number of epochs as needed
    total_loss = 0
    for batch in train_loader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss = loss.sum()
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{3}, Average Training Loss: {avg_train_loss:.4f}")

# Evaluation
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1]}
        labels = batch[2]
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predictions.extend(torch.argmax(probabilities, dim=1).tolist())
        true_labels.extend(labels.tolist())

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(true_labels, predictions))

# Calculate precision-recall curve and AUC
precision, recall, _ = precision_recall_curve(true_labels, predictions)
pr_auc = auc(recall, precision)
print(f"PR AUC: {pr_auc:.4f}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
2024-05-06 21:44:19.813193: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 21:44:19.813286: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has a

Epoch 1/3, Average Training Loss: 0.3415
Epoch 2/3, Average Training Loss: 0.2183
Epoch 3/3, Average Training Loss: 0.1133
Accuracy: 0.9424
              precision    recall  f1-score   support

           0       0.80      0.76      0.78      4420
           1       0.96      0.97      0.97     28580

    accuracy                           0.94     33000
   macro avg       0.88      0.87      0.87     33000
weighted avg       0.94      0.94      0.94     33000

PR AUC: 0.9798
